In [5]:
import numpy as np
import copy

#Make Blosum list
blosum_list = dict()
f2 = open('bioinfo2017/BLOSUM62.txt','r')
f2.readline()
for i in range(0,5):
    f2.next()
head_line = f2.next().strip().split()
len_head = len(head_line)
for line in f2:
    tmp_list = dict()
    for i in range(0,len_head):
        tmp_list[head_line[i]] = line.strip().split()[i+1]
    blosum_list[line.strip().split()[0]] = tmp_list
f2.close()

f1 = open('bioinfo2017/insulin_pep.uniprot.fa','r')
#Make prp_seq data
seq_data = dict()
for line in f1:
    if(line.startswith('>')):
        seq_h = line.strip().split(' ')[0].split('|')[2]
        seq_data[seq_h] = ''
    else:
        seq_data[seq_h] += line.strip()
f1.close()
        
#Find human pro_seq        
for key,value in seq_data.items():
    if key == 'INS_HUMAN':
        main_seq_key = key
        main_seq_value = value
        del seq_data[key]
        
#Make array for dynamic programming(row = Human, col = Other)
array_data = dict()
new_main_seq = dict()
tmp = 0
for key,value in seq_data.items():        
    array_data[key] = np.zeros([len(main_seq_value)+1, len(value)+1])
for key in seq_data.keys():
    new_main_seq[key] = main_seq_value[::-1]
    
#First row, col is -4*i
for key in array_data.keys():
    for row in range(0,array_data[key].shape[0]):
        array_data[key][row][0] += -4*row
    for col in range(0,array_data[key].shape[1]):
        array_data[key][0][col] += -4*col
        

#Make trace dict for new_pro_seq(with gap) & calculate and fill the cell
trace_data = copy.deepcopy(array_data)
for key in array_data.keys():
    for row in range(1,array_data[key].shape[0]):
        for col in range(1,array_data[key].shape[1]):
            match_score = int(blosum_list[main_seq_value[row-1]][seq_data[key][col-1]])
            max_value = max(array_data[key][row-1][col-1] + match_score ,array_data[key][row][col-1] -4,array_data[key][row-1][col]-4)
            array_data[key][row][col] += max_value            
            if (max_value == array_data[key][row-1][col-1] + match_score ):
                trace_data[key][row][col] += 1 #diagonal
            elif (max_value == array_data[key][row-1][col] -4):
                trace_data[key][row][col] += 3 #vertical
            else:
                trace_data[key][row][col] += 2 #horizontal    

#Make new_pro_seq from trace dict
new_seq = copy.deepcopy(seq_data)
for key in trace_data.keys():
    new_seq[key] = ''
    col = trace_data[key].shape[1]-2
    row  = trace_data[key].shape[0]-2
    while(col != 0):
        if (trace_data[key][row][col] == 1):
            new_seq[key] += seq_data[key][col]
            col -= 1
            row -= 1
        elif (trace_data[key][row][col] == 3):
            new_seq[key] += '-'
            row -= 1
        elif (trace_data[key][row][col] == 2):
            new_seq[key] += seq_data[key][col]
            col -= 1
for key in new_seq.keys():
    new_seq[key] += 'M'
            
#Print out
print 'name\tscore\n' ,'INS_HUMAN\n', main_seq_value
for key in array_data.keys():
    print key,'\t',array_data[key][-1][-1]
    print new_seq[key][::-1]

name	score
INS_HUMAN
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
INS_DANRE 	241.0
MAVWLQAGALLVLLVVSSVSTNPGT-PQHLCGSHLVDALYLVCGPTGFFYNPKR--DVEPLL-GFLPPKSAQETE-VADFAFKDHAELIRKRGIVEQCCHKPCSIFELQNYCN
INS_CHICK 	351.0
MALWIRSLPLLALLVFSGPGTSYAAANQHLCGSHLVEALYLVCGERGFFYSPKARRDVEQPLV-SSP-LR-GEA-GVLP-FQQEEYEKVKRGIVEQCCHNTCSLYQLENYCN
INS1_XENLA 	321.0
MALWMQCLPLVLVLFFSTPNT-EALVNQHLCGSHLVEALYLVCGDRGFFYYPKVKRDMEQALVSGPQDNELD---GMQ-LQPQEYQK-MK-RGIVEQCCHSTCSLFQLESYCN
INS1_MOUSE 	445.0
MALLVHFLPLLALLALWEPKPTQAFVKQHLCGPHLVEALYLVCGERGFFYTPKSRREVEDPQVEQLELGGSPG--DLQTLALEVARQKRGIVDQCCTSICSLYQLENYCN
INS_BOVIN 	453.0
MALWTRLRPLLALLALWPPPPARAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEGPQVGALELAGGPGAGG-----LEGPPQKRGIVEQCCASVCSLYQLENYCN
INS_PIG 	491.0
MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGG-L-QALALEGPPQKRGIVEQCCTSICSLYQLENYCN
